In [14]:
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt
import re

In [15]:
stopwords = ["a", "an", "the", "is", "to", "for", "in", "of", "and", "on", '']

In [16]:
def remove_stopwords(words, stopwords):
    return list(filter(lambda word: word not in stopwords, words))

In [17]:
def load_data(path: str, stopwords = ["a", "the", "is"]): 
    print('Importing data...')
    # format file into a list of tweets:
    lines = []
    with open(path, newline='\n') as file:
        lines = file.readlines()
    # take last column, take out punctuation, make lowercase, split into list of words
    for i, line in enumerate(lines):
        words_in_a_line = re.sub(r'!|:|,|\.|\|', '', line.split('|')[2]).lower().split(' ')
        lines[i] = remove_stopwords(words_in_a_line, stopwords)
    vocab = np.unique(np.concatenate(lines).flat)
    # creating data vector X:
    n = np.size(vocab)
    m = np.size(lines, axis=0)
    X = np.zeros((m,n))
    # populating X:
    for i, line in enumerate(lines):
        for word in line:
            j = np.where(vocab==word)[0]
            X[i,j] += 1

    print('X has {} examples and {} features in BOW format...'.format(m,n))
    return X, np.array(vocab)

In [18]:
X,vocab = load_data('data/cnnhealth.txt', stopwords)

Importing data...


/opt/anaconda3/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3213: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).shape[axis]


X has 4061 examples and 13685 features in BOW format...


In [19]:
lines = []
with open('data/cnnhealth.txt', newline='\n') as file:
    lines = file.readlines()

In [26]:
def generate_statistics(X,vocab):
    print("Dataset Statistics:")
    m,n = np.shape(X)
    num_words = int(np.sum(X))
    print("# Number of Tweets: {}".format(m))
    print("# Number of Words (Total): {}".format(num_words))
    print("# Number of Tokens: {}".format(n))
    print("# Average Number of Words per Tweet: {}".format(round((num_words/m),3)))
    print("# Top 5 Legal Tokens: {}".format(vocab[np.flip(np.argsort(np.sum(X, axis=0)))[0:5]]))

In [27]:
generate_statistics(X,vocab)

Dataset Statistics:
# Number of Tweets: 4061
# Number of Words (Total): 48233
# Number of Tokens: 13685
# Average Number of Words per Tweet: 11.877
# Top 5 Legal Tokens: ['you' 'rt' 'your' 'are' 'this']


In [23]:
# re.sub(r'!|:|,|.', '', line.split('|')[2]).lower().split(' ')

In [24]:
lines[0].split('|')[2]

'An abundance of online info can turn us into e-hypochondriacs. Or, worse, lead us to neglect getting the care we need http://cnn.it/1L1t1Fv\r\n'